<a href="https://colab.research.google.com/github/cbeckler/final_project/blob/kp_etl/Random_forest_NaN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu b

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-12 04:27:08--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2023-01-12 04:27:08 (11.2 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
# start spark session 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("random_forest").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#connect to database

import pandas as pd

conn_path = "/content/drive/MyDrive/data/db_conn_info.csv"

conn_info = pd.read_csv(conn_path)

db_name = conn_info['db_name'][0]

db_pw = conn_info['password'][0]

db_host = conn_info['host'][0]

db_port = conn_info['port'][0]

db_user = conn_info['user'][0]

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(f'postgresql://{db_name}:{db_pw}@{db_host}:{db_port}/{db_user}')

In [ ]:
# import model dataset

df = pd.read_sql("select * from null_model_dataset", engine)

df.head()

,index,FIPS,recent_trend,PM25_max_pred,PM25_med_pred,PM25_mean_pred,O3_max_pred,O3_med_pred,O3_mean_pred,PM25_max_pred_2001,...,O3_med_pred_2005,O3_med_pred_2006,O3_med_pred_2007,O3_med_pred_2008,O3_med_pred_2009,O3_med_pred_2010,O3_med_pred_2011,O3_med_pred_2012,O3_med_pred_2013,O3_med_pred_2014
0,0,0,falling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1001,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1003,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1005,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1007,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2672 entries, 0 to 2671
Data columns (total 93 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                2672 non-null   int64  
 1   FIPS                 2672 non-null   int64  
 2   recent_trend         2672 non-null   object 
 3   PM25_max_pred        2672 non-null   float64
 4   PM25_med_pred        2672 non-null   float64
 5   PM25_mean_pred       2672 non-null   float64
 6   O3_max_pred          2672 non-null   float64
 7   O3_med_pred          2672 non-null   float64
 8   O3_mean_pred         2672 non-null   float64
 9   PM25_max_pred_2001   2672 non-null   float64
 10  PM25_max_pred_2002   2672 non-null   float64
 11  PM25_max_pred_2003   2672 non-null   float64
 12  PM25_max_pred_2004   2672 non-null   float64
 13  PM25_max_pred_2005   2672 non-null   float64
 14  PM25_max_pred_2006   2672 non-null   float64
 15  PM25_max_pred_2007   2672 non-null   f

In [ ]:
col_list = [name for name in df.columns]

col_list = col_list[3:]

for col in col_list:

  df[col].fillna(df[col].mean(), inplace=True)

In [ ]:
df

,index,FIPS,recent_trend,PM25_max_pred,PM25_med_pred,PM25_mean_pred,O3_max_pred,O3_med_pred,O3_mean_pred,PM25_max_pred_2001,...,O3_med_pred_2005,O3_med_pred_2006,O3_med_pred_2007,O3_med_pred_2008,O3_med_pred_2009,O3_med_pred_2010,O3_med_pred_2011,O3_med_pred_2012,O3_med_pred_2013,O3_med_pred_2014
0,0,0,falling,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
1,1,1001,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
2,2,1003,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
3,3,1005,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
4,4,1007,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,2652,56003,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
2668,2653,56005,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
2669,2654,56007,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843
2670,2655,56009,stable,10.949364,10.546574,10.538006,41.736391,40.679811,40.693274,12.596511,...,41.988971,41.967347,42.174495,40.651883,38.208309,41.13262,41.485925,41.112726,38.688977,38.049843


In [ ]:
import numpy as np
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
from imblearn.over_sampling import SMOTE

In [ ]:
X = df.drop(['index', 'FIPS', 'recent_trend'], axis=1)
y = df['recent_trend']

In [ ]:
y.value_counts()

stable     2429
falling     200
rising       43
Name: recent_trend, dtype: int64

In [ ]:
#Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1,
                                                    stratify=y)
scaler =StandardScaler().fit(X_train)

X_train_scaled =scaler.transform(X_train)
X_test_scaled =scaler.transform(X_test)

In [ ]:
# Resampling training data with SMOTE
X_resampled, y_resampled = SMOTE(
    random_state=1,sampling_strategy='auto').fit_resample(
    X_train_scaled, y_train)

print(f'y_resampled: {Counter(y_resampled)}')

y_resampled: Counter({'stable': 1822, 'falling': 1822, 'rising': 1822})


In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
brc=BalancedRandomForestClassifier(random_state=1, n_estimators=100).fit(X_train_scaled, y_train)

In [ ]:
y_pred= brc.predict(X_test_scaled)

In [ ]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)


0.40232140182716786

In [ ]:

cm=confusion_matrix(y_test, y_pred)
cm

array([[ 35,   9,   6],
       [  7,   4,   0],
       [407, 113,  87]])

In [ ]:
brc_df=pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
brc_df

,Prediction,Actual
0,stable,stable
1,stable,stable
2,stable,stable
3,stable,stable
4,stable,stable
...,...,...
663,stable,stable
664,stable,stable
665,stable,stable
666,stable,stable


In [ ]:
print(brc_df.Prediction.value_counts())
print(brc_df.Actual.value_counts())

stable     664
falling      4
Name: Prediction, dtype: int64
stable     607
falling     50
rising      11
Name: Actual, dtype: int64


In [ ]:
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

    falling       0.08      0.70      0.33      0.14      0.48      0.24        50
     rising       0.03      0.36      0.81      0.06      0.54      0.28        11
     stable       0.94      0.14      0.90      0.25      0.36      0.12       607

avg / total       0.86      0.19      0.86      0.24      0.37      0.13       668

